***Important*** DO NOT CLEAR THE OUTPUT OF THIS NOTEBOOK AFTER EXECUTION!!!

In [23]:
# if the following command generates an error, you probably didn't enable 
# the cluster security option "Allow API access to all Google Cloud services"
# under Manage Security → Project Access when setting up the cluster
!gcloud dataproc clusters list --region us-central1

NAME          PLATFORM  PRIMARY_WORKER_COUNT  SECONDARY_WORKER_COUNT  STATUS   ZONE           SCHEDULED_DELETE
cluster-3b89  GCE       4                                             RUNNING  us-central1-a


# Imports & Setup

In [24]:
!pip install -q google-cloud-storage==1.43.0
!pip install -q graphframes

In [25]:
import pyspark
import sys
from collections import Counter, OrderedDict, defaultdict
import itertools
from itertools import islice, count, groupby
import pandas as pd
import os
import re
from operator import itemgetter
import nltk
import time
from nltk.stem.porter import *
from nltk.corpus import stopwords
from time import time
from pathlib import Path
import pickle
import pandas as pd
from google.cloud import storage
import math
import builtins as pybuiltins


import hashlib
def _hash(s):
    return hashlib.blake2b(bytes(s, encoding='utf8'), digest_size=5).hexdigest()

nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [26]:
# if nothing prints here you forgot to include the initialization script when starting the cluster
!ls -l /usr/lib/spark/jars/graph*

-rw-r--r-- 1 root root 247882 Mar  3 09:53 /usr/lib/spark/jars/graphframes-0.8.2-spark3.1-s_2.12.jar


In [27]:
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark import SparkContext, SparkConf, SparkFiles
from pyspark.sql import SQLContext
from graphframes import *

In [28]:
spark

In [29]:
# Put your bucket name below and make sure you can access it without an error
bucket_name = '207219783opt' 
full_path = f"gs://{bucket_name}/"
paths=[]

client = storage.Client()
blobs = client.list_blobs(bucket_name)
for b in blobs:
    if b.name != 'graphframes.sh' and not b.name.startswith("bucket_title") and not b.name.startswith("bucket_body") and not b.name.startswith("bucket_anchor"):
        paths.append(full_path+b.name)

In [30]:
parquetFile = spark.read.parquet(*paths)
# doc_title_pairs = parquetFile.select("title", "id").rdd
# doc_anchor_pairs = parquetFile.select("anchor_text", "id").rdd
# doc_body_pairs = parquetFile.select("text", "id").rdd

In [31]:
# if nothing prints here you forgot to upload the file inverted_index_gcp.py to the home dir
%cd -q /home/dataproc
!ls inverted_index_gcp.py

inverted_index_gcp.py


In [32]:
# adding our python module to the cluster
sc.addFile("/home/dataproc/inverted_index_gcp.py")
sys.path.insert(0,SparkFiles.getRootDirectory())

24/03/03 13:36:54 WARN SparkContext: The path /home/dataproc/inverted_index_gcp.py has been added already. Overwriting of added paths is not supported in the current version.


In [33]:
from inverted_index_gcp import InvertedIndex

**Project**


In [34]:
# stop words
english_stopwords = frozenset(stopwords.words('english'))
corpus_stopwords = ['category', 'references', 'also', 'links', 'extenal', 'see', 'thumb', 'became', 'may']
RE_WORD = re.compile(r"""[\#\@\w](['\-]?\w){2,24}""", re.UNICODE)
all_stopwords = english_stopwords.union(corpus_stopwords)


# tokenize and stemming
def tokenize_stemming(text):
    list_token = []
    stemmer = PorterStemmer()
    tokens = [token.group() for token in RE_WORD.finditer(text.lower())]
    for token in tokens:
        if token not in english_stopwords:
            list_token.append(stemmer.stem(token))
    return list_token


In [35]:
# tf
# normalizion by max frequency term
def term_frequency(text, id):
    tokens = tokenize_stemming(text)

    # Count the frequency of each word that is not a stopword
    word_freq = {}
    doc_size = 0

    for token in tokens:
        doc_size += 1
        if token not in all_stopwords:
            if token not in word_freq:
                word_freq[token] = 1
            else:
                word_freq[token] += 1

    lst_tuples = [(token, (id, freq)) for token, freq in word_freq.items()]
    return lst_tuples


def reduce_word_counts(unsorted_pl):
  sorted_pl = sorted(unsorted_pl, key=lambda x: x[0])
  return sorted_pl

# df
def calculate_df(postings):
  rdd_token_doc_freq = postings.map(lambda x: (x[0], len(x[1])))
  return rdd_token_doc_freq


# doc len
def doc_lenght(text, doc_id):
  tokens = tokenize_stemming(text)
  return((doc_id,len(tokens)))


In [36]:
NUM_BUCKETS = 124
def token2bucket_id(token):
  return int(_hash(token),16) % NUM_BUCKETS

def partition_postings_and_write(postings, index_name):
    bucketed_postings = postings.map(lambda x: (token2bucket_id(x[0]), [(x[0], x[1])]))         
    grouped_postings = bucketed_postings.reduceByKey(lambda x, y: x + y)                        
    posting_locations = grouped_postings.map(lambda x: InvertedIndex.write_a_posting_list(x, f'bucket_{index_name}', bucket_name))   

    return posting_locations


In [37]:
def create_norm_dict(index, postings):
    tf_idf_filtered = postings.map(lambda x: (x[0], [(tup[0], (tup[1] / index.doc_len[tup[0]]) * math.log2(index.N / index.df[x[0]])) for tup in x[1]]))
    
    # Calculate the sum of squared TF-IDF values for each document and collect as a dictionary
    norm = tf_idf_filtered.flatMap(lambda x: [(y[0], y[1] ** 2) for y in x[1]]).reduceByKey(lambda x, y: x + y).collectAsMap()
    
    return norm

In [38]:
def create_index(index_name, doc_pairs):

    word_counts = doc_pairs.flatMap(lambda x: term_frequency(x[0], x[1]))
#     word_counts_withNotSize = word_counts.map(lambda x: (x[0], (x[1][0], x[1][1])))
    postings = word_counts.groupByKey().mapValues(reduce_word_counts)
    postings_filtered = postings.filter(lambda x: len(x[1])>20)
    
    dl_rdd = doc_pairs.map(lambda x: doc_lenght(x[0], x[1]))
    w2df = calculate_df(postings_filtered)
    w2df_dict = w2df.collectAsMap()
    
    
    _ = partition_postings_and_write(postings_filtered, index_name).collect()

    super_posting_locs = defaultdict(list)
    for blob in client.list_blobs(bucket_name, prefix=f'bucket_{index_name}'):
        if not blob.name.endswith("pickle"):
            continue
        with blob.open("rb") as f:
            posting_locs = pickle.load(f)
            for k, v in posting_locs.items():
                super_posting_locs[k].extend(v)
                
    doc_title_dict = {v: k for k, v in doc_pairs.collectAsMap().items()}

                
    #################################
    # Create inverted index instance
    inverted = InvertedIndex()
    inverted.df = w2df_dict
    inverted.N = doc_pairs.count()
    inverted.doc_len = dict(dl_rdd.collect())
    inverted.posting_locs = super_posting_locs
    inverted.doc_id_title = doc_title_dict
    inverted.norm = create_norm_dict(inverted, postings_filtered)
    
    inverted.write_index('.', f'index_{index_name}')
    index_src = f"index_{index_name}.pkl"
    index_dst = f'gs://{bucket_name}/bucket_{index_name}/{index_src}'
    !gsutil cp $index_src $index_dst
    
    # return inverted

In [39]:
# weight = tf*idf
def calculate_tf_idf(index, term, tf):
  tf_idf = tf* (math.log2(index.N/index.df[term]))
  return tf_idf


def cosine_similarity(query, index):
    """ Returns: {doc_id:cosine score} """

    dict_cosine_sim = defaultdict(float)
    doc_weights_dict = defaultdict(float)
    query_dict = dict(term_frequency(query ,0))

    # create lists of keys and values from the query_dict
    query_list_keys = list(query_dict.keys())
    query_list_values = list(query_dict.values())

    # create list of items from index.df dictionery keys
    index_df_list_keys = list(index.df.keys())

    for term in query_list_keys:
      w_term_query = (query_dict[term][1]/len(query))* math.log2(index.N/index.df[term])

      if term in index_df_list_keys:
        posting_list = index.read_a_posting_list(".", term, bucket_name)
        
        for doc_id, freq in posting_list:  
          w_term_doc = calculate_tf_idf(index, term, freq/index.doc_len[doc_id])
          dict_cosine_sim[doc_id] += (w_term_doc)*(w_term_query)
        
    for doc_id in list(dict_cosine_sim.keys()):
      Word_doc_id__weight = index.norm[doc_id]
      dict_cosine_sim[doc_id] /= ( math.sqrt(pybuiltins.sum(value[1] ** 2 for value in query_list_values) * Word_doc_id__weight ))



    sorted_docs = sorted(dict_cosine_sim.items(), key=lambda x: x[1], reverse=True)
    top_100_docs = sorted_docs[:100]

    return top_100_docs

    

    

In [40]:
# def BM25(query, index, k, b, avg_doclen):
#     bm_score = {}
#     # create lists of keys and values from the query_dict
#     query_list_keys = list(query_dict.keys())
#     query_list_values = list(query_dict.values())
#     # create list of items from index.df dictionery keys
#     index_df_list_keys = list(index.df.keys())
    
#     idf_dict = calc_idf(query, N, index)
    
#     for term in query_list_keys:
#         if term in index_df_list_keys:
#             posting_list = index.read_a_posting_list(".", term, bucket_name)
#             for doc_id, freq in posting_list:
#                 B = 1 - b + b * (index.dl[doc_id] / avg_doclen)
#                 idf = idf_dict[term]
#                 tf = freq / index.dl[doc_id]
#                 if doc_id in bm_score:
#                     bm_score[doc_id] += (idf * freq * (k + 1)) / ((freq + B * k))
#                 else:
#                     bm_score[doc_id] = (idf * freq * (k + 1)) / ((freq + B * k))
    
#     return dict(sorted(bm_score.items(), key=lambda x: x[1], reverse=True))
            

In [ ]:
def search_title(inverted):
    res = []
    return res

def search_anchor(inverted):
    res = []
    return res


def search_res(inverted_title, inverted_body, inverted_anchor, query):
    start_time = time.time()  # Record the start time

    score_title = cosine_similarity(query, inverted_title)
    score_body = cosine_similarity(query, inverted_body)
    score_anchor = cosine_similarity(query, inverted_anchor)

    res_dict = defaultdict(float)
    
    for doc_id, score in score_title:
        res_dict[doc_id] += score * 7/16
    
    for doc_id, score in score_body:
        res_dict[doc_id] += score * 1/16

    for doc_id, score in score_anchor:
        res_dict[doc_id] += score * 7/16


    sorted_docs = sorted(res_dict.items(), key=lambda x: x[1], reverse=True)

    # Select the top 100 documents
    top_100_docs = sorted_docs[:100]

    end_time = time.time()  # Record the end time
    execution_time = end_time - start_time  # Calculate the execution time
    print("Function execution time:", execution_time, "seconds")
    
    # Return a list of tuples containing the document ID and its title
    result = [(doc_id, inverted_title.doc_id_title[doc_id]) for doc_id, _ in top_100_docs]


In [ ]:
# create_index("title", doc_title_pairs)
# create_index("body", doc_body_pairs)

# idx_title = InvertedIndex.read_index("bucket_title", 'index_title', bucket_name)
# idx_title = InvertedIndex.read_index("bucket_title", 'index_title', bucket_name)


In [ ]:
# query = "genetics"
# query = "Who is considered the"
# query = "economic"
# res_t = search(idx_title, query)
# res_b = search(idx_body, query)

